In [1]:
#-*- coding:utf-8 -*-
from commonTool import *
from config import *

# Parameter 설정
set_enc_param(encParamKey)

print('enc_param', enc_param_value)
print('codeFile', codePathFile)
print('outputFolder', outputRawPath, flush=True)


# https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage --> 메뉴 중 상장법인 목록
# 위에서 Excel로 내려 받은 파일임
# (code 목록, 업종 목록, 업종별 대표 코드 목록)을 반환.
def getCompanyCodeFromFile():
    codes = []
    types = []
    referCodes = []
    referChecker = {}

    fileCodes = open(codePathFile, 'r', encoding='utf-8')
    line = fileCodes.readline() # Title
    
    while True:
        line = fileCodes.readline()
        if not line:
            break

        line = line.strip()
        if len(line) <= 0:
            continue

        items = line.split('\t')
        
        codes.append(items[1])
        types.append(items[2])
        
        if not items[2] in referChecker:
            referChecker[items[2]] = items[1]
            referCodes.append(items[1])

    fileCodes.close()
    
    return (codes, types, referCodes)



# 데이터명 분석을 위한 데이터 페치
def doFetchDataNames(referenceCodes, outPath, fetchType, funcGet):
    print(fetchType, 'doing...', flush=True)
    deli = '\t'
    # keyNames = ['ACCODE', 'ACC_NM', 'UNT_TYP', 'P_ACCODE', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    valueColumns = ['DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']

    out = open(outPath + fetchType + '-accNames.txt', 'w', encoding='utf-8')
    out.write('PCODE' + deli + 'ACCODE' + deli + 'NAME' + deli + 'P_ACCODE\n')

    for code in referenceCodes:
        # print('fetching', code, flush=True)
        resp = funcGet(code)

        if resp.status_code == 200:
            # print(resp.text, flush=True)
            obj = json.loads(resp.text)
            if obj is None: continue
            
            data = obj['DATA']
            if data is None:
                continue

            accodeMap = {}
            
            for rec in data:
                if not rec['ACCODE'] in accodeMap:
                    accNm = rec['ACC_NM'].lstrip('.').lstrip('*')
                    accodeMap[rec['ACCODE']] = { 'name': accNm, 'pCode': rec['P_ACCODE'] }
                    
            # 추출한 데이터 명칭 저장
            
            for key in accodeMap.keys():
                elem = accodeMap[key]
                accNm = elem['name']
                
                out.write(code)
                out.write(deli)
                out.write(key)
                out.write(deli)
                out.write(accNm if elem['pCode'] is None else accodeMap[elem['pCode']]['name'] + '/' + accNm)
                out.write(deli)
                out.write('' if elem['pCode'] is None else elem['pCode'])
                out.write('\n')
            
        else:
            print('fetch error ', code, resp.status_code)

    out.close()
    print(fetchType, 'done...', flush=True)


enc_param UWtSVjg4a0tjL3psTkRScjMzdDFtQT09
codeFile D:\work\jupyter\krx\resource\companyInfo-20210530.txt
outputFolder D:\work\jupyter\krx\output\


In [ ]:
# 1. 데이터를 가져올 회사 정보 가져 오기
(codes, types, referCodes) = getCompanyCodeFromFile()


In [ ]:
# 2. 업종별 재무 데이터 항목 가져와 파일에 저장하기
# 최종 필요한 파일은 Step 3에서 생성되는 파일로 여기서 크롤힝하는 데이터는 임시 폴더에 넣음
print('reference codes count: ', len(referCodes))
fetchFunc = { 'annualBS': fetchFinacialData, 'annualCF': fetchCashFlowData, 'annualPL': fetchProfitLossData }

for dataType in fetchFunc.keys():
    doFetchDataNames(referCodes, temporaryPath, dataType, fetchFunc[dataType])


In [ ]:
# 3. 2에서 가져온 재무 항목 정리
# (TBD) Crawlego 스크립트 실행 필요 (AC-CODE-SAVE.xml)
# ./resource 폴더 내 accCodes-XX.txt 파일 생성 (XX: BS, CF, PL)

#

In [4]:
# 4. 3에서 분석된 재무 항목을 업종별 컬럼목록 객체로 변환하여 반환
columnNameFile = resourceDir + 'accCodes-BS.txt'

file = open(columnNameFile, 'r', encoding='utf-8')
line = file.readline() # Title

columnsMap = {}


while True:
    line = file.readline()
    if not line:
        break

    line = line.strip()
    if len(line) <= 0:
        continue

    # 업종구분, ACCODE, NAME, P_ACCODE
    items = line.split('\t')
    
    if not items[0] in columnsMap:
        columnsMap[items[0]] = { 'accMap':{}, 'accNames':[] }
        
    theItem = columnsMap[items[0]]
    
    theItem['accMap'][items[1]] = len(theItem['accNames'])
    theItem['accNames'].append(items[2])

file.close()

print(columnsMap)


{'보험업;재 보험업;': {'accMap': {'110000': 0, '110010': 1, '110020': 2, '110030': 3, '110090': 4, '110100': 5, '110110': 6, '114470': 7, '110120': 8, '110130': 9, '190010': 10, '110180': 11, '110190': 12, '190020': 13, '111680': 14, '111740': 15, '110220': 16, '110230': 17, '190030': 18, '110280': 19, '110290': 20, '110300': 21, '190040': 22, '114480': 23, '110330': 24, '190050': 25, '110380': 26, '110390': 27, '190060': 28, '114490': 29, '110420': 30, '111800': 31, '114510': 32, '190070': 33, '190080': 34, '190090': 35, '190100': 36, '190110': 37, '190120': 38, '190125': 39, '190130': 40, '190140': 41, '110870': 42, '190150': 43, '190160': 44, '190170': 45, '190180': 46, '190190': 47, '190200': 48, '190210': 49, '190220': 50, '111170': 51, '190230': 52, '190240': 53, '111270': 54, '111280': 55, '111290': 56, '190250': 57, '190260': 58, '111870': 59, '190270': 60, '111450': 61, '111460': 62, '111470': 63, '190280': 64, '190290': 65, '190300': 66, '190310': 67, '111570': 68, '111580': 69, '11

In [ ]:
# 5. 회사별 재무제표 가져오기 업종에 따라 다른 파일에 저장됨.


def doFetch(codeFile, outPath, fetchType, funcGet):
    print(fetchType, 'doing...', flush=True)
    deli = '\t'
    # keyNames = ['ACCODE', 'ACC_NM', 'UNT_TYP', 'P_ACCODE', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    valueColumns = ['DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5']
    
    f = open(codeFile, 'r')
    
    accCheck = {}
    accNames = []
    accItems = []
    
    dataSet = []
    
    noLine = 0
    while True:
        line = f.readline()
        if not line:
            break

        noLine += 1
        code = line.strip()
        if len(code) <= 0:
            continue

        print('fetching', code, flush=True)
        resp = funcGet(code)

        if resp.status_code == 200:
            # print(resp.text, flush=True)
            obj = json.loads(resp.text)
            if obj is None: continue
            
            # DATA1 ~ DATA5의 기준 년월
            yearName = [T[0:T.find('<')] for T in obj['YYMM']][0:len(valueColumns)]

            data = obj['DATA']
            if data is None:
                continue

            items = [extendList([code, title], len(accNames) + 2) for title in yearName] # DATA1, ..., DATA4
            
            for rec in data:
                if not rec['ACCODE'] in accCheck:
                    accCheck[rec['ACCODE']] = len(accNames)
                    accNm = rec['ACC_NM'].lstrip('.').lstrip('*')
                    accNames.append(accNm if rec['P_ACCODE'] is None else accNames[accCheck[rec['P_ACCODE']]] + '/' + accNm)
                    accItems.append([rec['ACCODE'], accNames[-1], rec['P_ACCODE']])
                    items = [extendList(T, len(accNames) + 2) for T in items]

                # 저장할 위치
                valCol = accCheck[rec['ACCODE']] + 2
                
                for c in range(0, len(items)):
                    items[c][valCol] = rec[valueColumns[c]]

            # 결과 저장용 배열에 추가
            dataSet.extend(items)
        else:
            print('fetch error ', code, resp.status_code)

    # 추출한 데이터 명칭 저장
    out = open(outPath + fetchType + '-accNames.txt', 'w', encoding='utf-8')
    out.write('ACCODE' + deli + 'NAME' + deli + 'P_ACCODE\n')
    for elem in accItems:
        out.write(elem[0])
        out.write(deli)
        out.write(elem[1])
        out.write(deli)
        out.write('' if elem[2] is None else elem[2])
        out.write('\n')
        
    out.close()
    
    
    # 추출한 데이터 저장
    out = open(outPath + fetchType + '.txt', 'w', encoding='utf-8')
    
    # 컬럼명 저장
    out.write('P_CODE' + deli + 'TERM')
    
    for key in accNames:
        out.write(deli)
        out.write(key)
    out.write('\n')
    
    for rec in dataSet:
        if rec[2] is None:
           continue
        
        for i in range(0, len(rec)):
            v = rec[i]
            if i > 0: out.write(deli)
            out.write('' if v is None else str(v))
        out.write('\n')
        
    out.close()
    f.close()
    
    print(fetchType, 'done...', flush=True)


# Main Proc

# fetchFunc = { 'TestOut': fetchFinacialData }
             
fetchFunc = { 'annualBS': fetchFinacialData, 'annualCF': fetchCashFlowData, 'annualPL': fetchProfitLossData,
    'quarterBS': fetchQuaterFinacialData, 'quarterCF': fetchQuaterCashFlowData, 'quarterPL': fetchQuaterProfitLossData }

for dataType in fetchFunc.keys():
    doFetch(codePathFile, outputRawPath, dataType, fetchFunc[dataType])


In [5]:
typeName = 'BS'
testDict = {}

testDict['annual' + typeName] = 'hello'

print(testDict)



{'annualBS': 'hello'}
